### API

In [3]:
#!/usr/bin/env python3
import pandas as pd

# ---- FILE PATHS ----
WEATHER_CSV = "C:\\Users\\roosd\\Downloads\\econometrie jaar 3\\TIC\\Spring2026-TIC\\data\\combined_daily_weather_data.csv"
SOYBEAN_CSV = "C:\\Users\\roosd\\Downloads\\econometrie jaar 3\\TIC\\Spring2026-TIC\\data\\daily_soybean_prices.csv"
CORN_CSV = "C:\\Users\\roosd\\Downloads\\econometrie jaar 3\\TIC\\Spring2026-TIC\\data\\daily_corn_prices.csv"
OUT_CSV = "merged_prices_weather.csv"

# ---- 1) Load weather and clean datetime -> date ----
weather = pd.read_csv(WEATHER_CSV)

# Parse timezone-aware timestamps, normalize to date, then drop timezone (IMPORTANT for merge)
weather["date"] = (
    pd.to_datetime(weather["date"], errors="coerce", utc=True)
      .dt.normalize()
      .dt.tz_convert(None)   # <-- key fix: datetime64[ns, UTC] -> datetime64[ns]
)

# Keep only 2005-2025 inclusive
weather = weather[(weather["date"] >= "2005-01-01") & (weather["date"] <= "2025-12-31")].copy()

# ---- 2) Load soybean + corn, clean the extra metadata row ----
def load_price_csv(path: str, prefix: str) -> pd.DataFrame:
    df = pd.read_csv(path)

    # Drop rows where Date is missing (this removes the ",ZS=F,ZS=F," line)
    df = df[df["Date"].notna()].copy()

    # Parse Date as date-only (naive datetime64[ns])
    df["date"] = pd.to_datetime(df["Date"], errors="coerce").dt.normalize()

    # Drop original Date column
    df = df.drop(columns=["Date"])

    # Rename columns to avoid collisions after merge
    df = df.rename(columns={
        "Close": f"{prefix}_Close",
        "Volume": f"{prefix}_Volume",
        "log_return": f"{prefix}_log_return"
    })

    # Keep only relevant rows
    df = df[df["date"].notna()].copy()
    return df

soy = load_price_csv(SOYBEAN_CSV, "soybean")
corn = load_price_csv(CORN_CSV, "corn")

# ---- 3) Merge soybean + corn on date ----
prices = pd.merge(soy, corn, on="date", how="outer")

# ---- 4) Merge prices with weather on date ----
merged = pd.merge(prices, weather, on="date", how="left")

# Sort and write
merged = merged.sort_values("date").reset_index(drop=True)
merged.to_csv(OUT_CSV, index=False)

print("Done")
print("Dtypes check:")
print("  prices['date'] :", prices['date'].dtype)
print("  weather['date']:", weather['date'].dtype)
print(f"Weather rows (2005-2025): {len(weather):,}")
print(f"Soy rows: {len(soy):,} | Corn rows: {len(corn):,}")
print(f"Final merged rows: {len(merged):,}")
print(f"Saved: {OUT_CSV}")


Done
Dtypes check:
  prices['date'] : datetime64[ns]
  weather['date']: datetime64[ns]
Weather rows (2005-2025): 22,854
Soy rows: 5,305 | Corn rows: 5,303
Final merged rows: 15,801
Saved: merged_prices_weather.csv
